# Prediction using GPR data

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import geopandas as gpd
from mpl_toolkits.mplot3d import Axes3D 

In [2]:
path = '/Users/paw_paw/Library/CloudStorage/GoogleDrive-keyuqing597@gmail.com/我的云端硬盘/PC5287_M.SC Coursework Thesis for Physics And Technology/data set'

$$
\langle T_{ij} \rangle = T_i\cdot \frac{m_i\cdot n_j}{(m_i+s_{ij})(m_i+n_j+s_{ij})}
$$
in which:
$$
T_i=\sum_{j\neq i}T_{ij}
$$
and $\langle T_{ij} \rangle$ means tha mean value of $T_{ij}$ within a period of time. 

## Import population flux data

In [3]:
T_i_bus = pd.read_csv(path + '/Total_bus_flux2019.csv').drop(columns='Unnamed: 0')

T_ij_bus= pd.read_csv(path+'/Total_bus_flux_ij2019.csv').drop(columns='Unnamed: 0')

T_i_MRT = pd.read_csv(path + '/Total_MRT_flux2019.csv').drop(columns='Unnamed: 0')

T_ij_MRT = pd.read_csv(path + '/Total_MRT_flux_ij2019.csv').drop(columns='Unnamed: 0')

## import the geometry data 

In [6]:
#read KML file
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [7]:
pa = gpd.read_file(path+'/master-plan-2019-planning-area-boundary-no-sea/planning-boundary-area.kml', \
                            driver='KML')
pa = pa.to_crs(3414)

def extract_desc_pa(desc_pa):
    return desc_pa.split('PLN_AREA_N')[1].split('<td>')[1].split('<')[0]

pa['PLN_AREA_N'] = pa.Description.apply(extract_desc_pa)

pa['centroid'] = pa.centroid

## import population (GPR) data 

In [21]:
GPR = pd.read_csv(path+'/SG_GPR_sorted.csv').drop(columns='Unnamed: 0')

In [23]:
from shapely import wkt

GPR['geometry'] = GPR['geometry'].apply(wkt.loads)
GPR = gpd.GeoDataFrame(GPR, crs='epsg:3414')

In [24]:
GPR

,GPR,GPR_NUM,PLN_AREA_N,SUBZONE_N,SHAPE_Area,POP_DENS,geometry
0,1.0,1.0,TUAS,TUAS VIEW EXTENSION,735026.472606,735026.472606,"POLYGON ((3606.500 26171.307, 3606.500 26171.3..."
1,1.0,1.0,TUAS,TUAS VIEW EXTENSION,345914.636037,345914.636037,"POLYGON ((3606.131 27006.302, 3606.500 26171.3..."
2,1.0,1.0,TUAS,TUAS VIEW EXTENSION,9411.081297,9411.081297,"POLYGON ((3606.131 27006.302, 2668.620 27006.3..."
3,1.0,1.0,TUAS,TUAS VIEW EXTENSION,966310.427138,966310.427138,"POLYGON ((3240.090 28525.774, 3856.615 28295.2..."
4,1.0,1.0,TUAS,TUAS VIEW EXTENSION,8077.741680,8077.741680,"POLYGON ((3855.740 28295.878, 3856.615 28295.2..."
...,...,...,...,...,...,...,...
33112,1.4,1.4,MARINE PARADE,KATONG,288.225272,403.515381,"POLYGON ((35964.658 32631.632, 35952.880 32622..."
33113,1.4,1.4,MARINE PARADE,KATONG,130.410105,182.574148,"POLYGON ((35930.834 32637.795, 35925.593 32633..."
33114,1.4,1.4,MARINE PARADE,KATONG,164.929047,230.900666,"POLYGON ((35945.784 32617.567, 35940.580 32613..."
33115,1.4,1.4,MARINE PARADE,KATONG,158.483763,221.877268,"POLYGON ((35940.580 32613.595, 35938.195 32611..."


In [25]:
Pop_dens = pd.DataFrame()

Pop_dens = GPR.groupby(GPR['PLN_AREA_N']).sum(numeric_only=True)["POP_DENS"].reset_index()

In [26]:
Pop_dens

,PLN_AREA_N,POP_DENS
0,ANG MO KIO,1.103508e+07
1,BEDOK,2.057210e+07
2,BISHAN,6.030975e+06
3,BOON LAY,1.032779e+07
4,BUKIT BATOK,9.928237e+06
5,BUKIT MERAH,1.095066e+07
6,BUKIT PANJANG,6.631214e+06
7,BUKIT TIMAH,4.795816e+06
8,CHANGI,1.815505e+06
9,CHOA CHU KANG,8.618132e+06


In [27]:
GPR['centroid']=GPR.centroid

define the function to get s_ij. 

In [28]:
GPR['centroids'] = GPR['centroid'].apply(lambda x: x.wkt).values

In [31]:
import shapely

In [29]:
GPR_centroid_pop = GPR.set_index(['centroids'])['POP_DENS'].to_dict()

In [32]:
def s_ij(pa_i, pa_j):
    r_ij = pa_i.distance(pa_j)
    s_ij = 0 
    gpr_ctr = shapely.wkt.loads(list(GPR_centroid_pop.keys())[0])
    
    for k in list(GPR_centroid_pop.keys()):
        if shapely.wkt.loads(k).distance(pa_j)<=r_ij:
            s_ij = s_ij + GPR_centroid_pop.get(k)
    return s_ij 

In [33]:
pa_centroid=pa.set_index(['PLN_AREA_N'])['centroid'].to_dict()

In [34]:
#test s_ij
s_ij(pa_centroid.get('NEWTON'),pa_centroid.get('TANGLIN'))

12836423.489157546

## Prediction

In [25]:
def get_T_i(pa):
    
    return T_i_bus.set_index(['ORIGIN_loc'])['TOTAL_TRIPS'].to_dict().get(pa)+\
T_i_bus.set_index(['ORIGIN_loc'])['TOTAL_TRIPS'].to_dict().get(pa)

In [80]:
def get_pop(pa):
    pa = str(pa)
    if Pop_dens.set_index(['PLN_AREA_N'])['POP_DENS'].to_dict().get(pa)!= None:
        return Pop_dens.set_index(['PLN_AREA_N'])['POP_DENS'].to_dict().get(pa)
    else:
        return float(0)

In [146]:
def func(pa_i, pa_j):
    T_i = get_T_i(pa_i)
    m_i = get_pop(pa_i)
    n_j = get_pop(pa_j)
    s = s_ij(pa_centroid.get(pa_i),pa_centroid.get(pa_j))
    try:
        return T_i*m_i*n_j/((m_i+s)*(m_i+n_j+s))
    except ZeroDivisionError: 
        return None

In [1]:
pd.concat([Pop_dens.PLN_AREA_N,pa.PLN_AREA_N]).drop_duplicates(keep = False).reset_index().drop(columns='index')

NameError: name 'pd' is not defined

In [112]:
pd.concat([T_i_bus.ORIGIN_loc,pa.PLN_AREA_N])\
.drop_duplicates(keep = False).reset_index().drop(columns='index')

,0
0,NORTH-EASTERN ISLANDS
1,SIMPANG
2,MARINA EAST
3,MARINA SOUTH
4,ORCHARD
5,OUTRAM
6,STRAITS VIEW
7,CHANGI BAY
8,WESTERN ISLANDS


In [113]:
pd.concat([T_i_MRT.ORIGIN_loc,pa.PLN_AREA_N])\
.drop_duplicates(keep = False).reset_index().drop(columns='index')

,0
0,CENTRAL WATER CATCHMENT
1,LIM CHU KANG
2,MANDAI
3,NORTH-EASTERN ISLANDS
4,BOON LAY
5,SELETAR
6,SIMPANG
7,SOUTHERN ISLANDS
8,TENGAH
9,MARINA EAST


We can see that GPR, bus and MRT flux data all have missing data more or less.    By looking into the background of these planning areas, we can find that they are all places that lack residents. 

In [88]:
def fix(non_fixed,non_fixed_col):
    temp = pd.concat([non_fixed[non_fixed_col],pa.PLN_AREA_N]).drop_duplicates(keep = False).\
    reset_index().drop(columns='index')
    temp[non_fixed_col] = pd.DataFrame(np.zeros(temp.shape[0]))
    return pd.concat([non_fixed,temp]).reset_index().drop(columns='index')

In [89]:
Pop_dens=fix(Pop_dens,'PLN_AREA_N')

In [90]:
Pop_dens

,PLN_AREA_N,POP_DENS
0,ANG MO KIO,1.103508e+07
1,BEDOK,2.057210e+07
2,BISHAN,6.030975e+06
3,BOON LAY,1.032779e+07
4,BUKIT BATOK,9.928237e+06
5,BUKIT MERAH,1.095066e+07
6,BUKIT PANJANG,6.631214e+06
7,BUKIT TIMAH,4.795816e+06
8,CHANGI,1.815505e+06
9,CHOA CHU KANG,8.618132e+06


In [137]:
temp = pd.concat([T_i_bus['ORIGIN_loc'],pa.PLN_AREA_N]).drop_duplicates(keep = False).\
    reset_index().drop(columns='index')

temp=temp.rename(columns={0:'ORIGIN_loc'})

temp['TOTAL_TRIPS'] = pd.DataFrame(np.zeros(temp.shape[0]))

T_i_bus=pd.concat([T_i_bus,temp]).reset_index().drop(columns='index')

T_i_bus

,ORIGIN_loc,TOTAL_TRIPS
0,ANG MO KIO,5753076.0
1,BEDOK,8076632.0
2,BISHAN,2680822.0
3,BOON LAY,186710.0
4,BUKIT BATOK,3588419.0
5,BUKIT MERAH,4666840.0
6,BUKIT PANJANG,2520233.0
7,BUKIT TIMAH,2222822.0
8,CENTRAL WATER CATCHMENT,168233.0
9,CHANGI,1164105.0


In [136]:
temp = pd.concat([T_i_MRT['ORIGIN_loc'],pa.PLN_AREA_N]).drop_duplicates(keep = False).\
    reset_index().drop(columns='index')

temp=temp.rename(columns={0:'ORIGIN_loc'})

temp['TOTAL_TRIPS'] = pd.DataFrame(np.zeros(temp.shape[0]))

T_i_MRT=pd.concat([T_i_MRT,temp]).reset_index().drop(columns='index')

T_i_MRT

,ORIGIN_loc,TOTAL_TRIPS
0,ANG MO KIO,2537780.0
1,BEDOK,2822888.0
2,BISHAN,1861313.0
3,BUKIT BATOK,2393861.0
4,BUKIT MERAH,4074769.0
5,BUKIT PANJANG,1522764.0
6,BUKIT TIMAH,1451151.0
7,CHANGI,593403.0
8,CHOA CHU KANG,2720252.0
9,CLEMENTI,1432139.0


In [150]:
for pa_i in Prediction['PA_name']:
    Prediction[pa_i]=Prediction['PA_name'].apply(func,args=([pa_i]))            

In [151]:
Prediction

,PA_name,BUKIT MERAH,BUKIT PANJANG,BUKIT TIMAH,CENTRAL WATER CATCHMENT,CHANGI,CHOA CHU KANG,CLEMENTI,HOUGANG,JURONG EAST,...,SINGAPORE RIVER,STRAITS VIEW,CHANGI BAY,MARINE PARADE,GEYLANG,PAYA LEBAR,WESTERN ISLANDS,WESTERN WATER CATCHMENT,WOODLANDS,YISHUN
0,BUKIT MERAH,4.666840e+06,1.136999e+04,3.769311e+04,0.0,3.480235e+03,1.131059e+04,4.901424e+04,2.719318e+04,4.798556e+04,...,67618.972741,134514.467679,0.0,2.650676e+04,3.542918e+04,6281.776852,214394.403657,32.899272,1.486710e+04,1.086539e+04
1,BUKIT PANJANG,1.230493e+04,2.520233e+06,1.364503e+05,0.0,8.407048e+02,2.211305e+05,6.398392e+04,8.487340e+03,3.919223e+04,...,1373.948154,642.986395,0.0,1.953350e+03,7.642319e+03,1765.153553,22769.765683,74.038916,9.609034e+04,2.587158e+04
2,BUKIT TIMAH,3.685087e+04,9.119385e+04,2.222822e+06,0.0,6.590896e+02,2.510611e+04,1.382593e+05,6.470556e+03,3.192913e+04,...,2895.347535,1321.086737,0.0,1.612321e+03,6.071609e+03,1351.377938,26973.849963,23.104799,2.220989e+04,1.013956e+04
3,CENTRAL WATER CATCHMENT,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
4,CHANGI,2.532054e+02,1.429601e+02,1.039617e+02,0.0,1.164105e+06,1.849335e+02,2.134934e+02,1.167394e+03,3.566532e+02,...,37.479620,70.044381,0.0,4.832779e+02,1.096958e+03,459.625992,680.698800,0.548468,3.455395e+02,3.644700e+02
5,CHOA CHU KANG,8.696216e+03,4.264176e+05,1.786515e+04,0.0,1.116033e+03,3.591399e+06,4.054148e+04,1.060257e+04,5.629717e+04,...,956.097940,666.595137,0.0,2.366021e+03,9.240693e+03,2268.167054,35694.963199,982.190486,1.930994e+05,2.993928e+04
6,CLEMENTI,6.925405e+04,1.439820e+05,4.841049e+05,0.0,1.516572e+03,7.928808e+04,3.606280e+06,1.317659e+04,7.670004e+05,...,4493.206177,2572.042950,0.0,4.491719e+03,1.542793e+04,2903.036361,292134.937609,118.650201,4.016707e+04,1.279406e+04
7,HOUGANG,5.578704e+04,1.655382e+04,1.475560e+04,0.0,3.126594e+04,1.797158e+04,1.852461e+04,6.003248e+06,2.711828e+04,...,10934.272187,9648.460998,0.0,3.972773e+04,4.026953e+05,553858.833984,59858.543510,43.053512,9.177479e+04,1.772212e+05
8,JURONG EAST,5.289473e+04,1.031333e+05,7.067010e+04,0.0,1.903902e+03,1.082831e+05,7.512737e+05,1.706031e+04,3.698546e+06,...,3625.427811,2404.684280,0.0,5.718059e+03,1.810467e+04,3694.762432,641236.152166,261.013403,4.269459e+04,1.256209e+04
9,JURONG WEST,3.500625e+04,1.329715e+05,3.571650e+04,0.0,3.506807e+03,3.924424e+05,2.288060e+05,3.320370e+04,7.554681e+05,...,3139.806154,2604.471023,0.0,7.374431e+03,2.833366e+04,7014.507096,374190.959289,11285.263966,1.268151e+05,2.514622e+04


In [159]:
Prediction.to_csv(path+'/Prediction2019_gpr.csv')